In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd
import ipfn

# Imports

In [3]:
userhome = os.path.expanduser('~')
userhome = userhome.replace('C:', 'D:')

In [4]:
infolder = os.path.join(userhome, r'OneDrive - Transport Systems Catapult\Modelling\Data\GIS\Boundaries\Lookups\OA')
infilepath = os.path.join(infolder, 'OA11_LSOA11_MSOA11_LAD11_EW_LUv2.csv')

lu = pd.read_csv(infilepath)

lookups = {}
lu = lu[lu.columns[3:]].drop_duplicates()
lu.set_index('MSOA11CD', inplace=True)
lookups['MSOA11CD'] = lu

In [5]:
infolder = r'inputs'
infile = r'tripends_msoas_2026_purpose_mode_time.csv'
infilepath = os.path.join(infolder, infile)

tripends = pd.read_csv(infilepath, index_col=[0, 1, 2, 3, 4])
tripends.head(2)

TripEnds
MSOA11CD  Purpose Mode TimePeriod TripType            
E02000001 HBO     Bus  AM         D         2325.83987
                                  O          910.49436

In [6]:
infile = r'tlds_purpose_mode.csv'
infilepath = os.path.join(infolder, infile)

tlds = pd.read_csv(infilepath, index_col=[0, 1, 2])
# Rename for consistency
tlds.index.names = ['Mode', 'Purpose', 'DistanceBand']
tlds.columns = ['GroupPercentage']
tlds.head(2)

GroupPercentage
Mode Purpose DistanceBand                 
Bus  HBO     [0.0, 1.0)           0.046845
             [1.0, 2.0)           0.252021

In [7]:
infile = r'distance_matrix_msoas_london.csv'
infilepath = os.path.join(infolder, infile)

distance_matrix = pd.read_csv(infilepath, index_col=[0, 1])
distance_matrix.head(2)

Distance_miles
orig_zoneID dest_zoneID                
E02000001   E02000001              0.00
            E02000576              0.68

In [8]:
#TODO improve intrazonal distance (from 0 to 1 mile)
distance_matrix.Distance_miles.replace(0, 1, inplace=True)

## Gravity Model

In [9]:
idxslc = pd.IndexSlice

In [10]:
beta = 0.1
cost_matrix = np.exp((-beta*distance_matrix.Distance_miles))

In [11]:
def create_seed_matrix(cost_matrix, origs):
    df = cost_matrix.div(cost_matrix.sum()).mul(origs.sum())
    return df

In [12]:
def balance_dests(origs, dests, balance_internal_zones=True, internal_zones=None, verbose=True):
    if not balance_internal_zones and len(internal_zones) > 0:
        # Exclude the internal zone dests from the sum
        external_zones_bool = ~dests.index.get_level_values(0).isin(internal_zones)
        dests_external = dests[external_zones_bool]
        dests_internal = dests[internal_zones]
        dests_tot = dests_external.sum()
        
        # The destinations of the internal zones must be excluded from the sum
        balance_factor = (origs.sum() - dests_internal.sum()) / dests_tot
        # Apply the factor only to the external zones
        dests[external_zones_bool] *= balance_factor
    
    else:
        dests_tot = dests.sum()
        balance_factor = origs.sum() / dests_tot
        dests *= balance_factor
    
    if verbose:
        if ((balance_factor >= 1.05) or (balance_factor <= 0.95)):
            print('Origs/Dests factor: {}'.format(balance_factor))
            
    return dests

In [13]:
def append_distance_band(trips, dists, bins):
    df = pd.merge(trips.to_frame(), dists, left_index=True, right_index=True)
    dist_bands = df.Distance_miles.transform(lambda x: pd.cut(x, bins, right=False))
    df['DistanceBand'] = dist_bands 
    return df

In [14]:
def get_distance_bins(grpd_dists_df):
    lbls = [l for l in grpd_dists_df.index]
    bins = []
    for lbl in lbls:
        tmp = lbl
        for ch in '[ ] ( )'.split():
            tmp = tmp.replace(ch, '')
        lower, upper = tmp.split(',')
        lower = float(lower)
        upper = float(upper)
        bins.append(lower)
        bins.append(upper)
    
    bins = list(set(bins))
    bins.sort()
    return bins

In [15]:
def to_interval_index(df, closed='left'):
    import re
    
    idx = df.index
    idx_name = idx.name
    
    p = re.compile('[-+]?\d*\.\d+|\d+')
    lower = [float((p.findall(interv)[0])) for interv in idx]
    upper = [float((p.findall(interv)[1])) for interv in idx]

    intervIdx = pd.IntervalIndex.from_arrays(lower, upper, closed=closed)
    
    df.index = intervIdx
    df.index.name = idx_name

    return df

In [16]:
def furness(df, aggrs, dims, **kwargs):
    
    conv_rate = kwargs['convergence_rate']
    max_iter = kwargs['max_iteration']
    weight_col = kwargs['weight_col']
    verbose = kwargs['verbose']
    
    df = df.reset_index().copy() 
    ipf = ipfn.ipfn(df, aggregates, dimensions, convergence_rate=conv_rate,
                    max_iteration=max_iter, weight_col=weight_col, verbose=verbose)
    df_ipf = ipf.iteration()
    
    return df_ipf
    

In [17]:
# Zones in Greenwich District
lu = lookups['MSOA11CD']
internal_zones = lu[lu.LAD11NM == 'Greenwich']
internal_zones.head()

,MSOA11NM,LAD11CD,LAD11NM
MSOA11CD,,,
E02000320,Greenwich 008,E09000011,Greenwich
E02000315,Greenwich 003,E09000011,Greenwich
E02000328,Greenwich 016,E09000011,Greenwich
E02000333,Greenwich 021,E09000011,Greenwich
E02000326,Greenwich 014,E09000011,Greenwich


In [21]:
# Filter the ipfn inputs
filter_modes = ['CarDriver', 'CarPass', 'Bus']

In [22]:
gtripends = tripends.groupby(level=['Purpose', 'Mode', 'TimePeriod'])

conv_rates = [0.05, 0.1, 0.15, 0.2, 0.3, 0.35]

ipfn_results = {}

for k, g in gtripends:
    purp , md, tp = k
    
    if md in filter_modes:
        xs_lvls = ['Purpose', 'Mode', 'TimePeriod', 'TripType']
        origs = g.xs([purp, md, tp, 'O'], level=xs_lvls).squeeze()
        dests = g.xs([purp, md, tp, 'D'], level=xs_lvls).squeeze()
        origs.index.name = 'orig_zoneID'
        dests.index.name = 'dest_zoneID'

        dests_orig = dests.copy()
        dests = balance_dests(origs, dests, balance_internal_zones=False,
                              internal_zones=internal_zones.index)

        # Keep a factor of the modification
        dests_factor = dests.div(dests_orig)
        
        seed_matrix = create_seed_matrix(cost_matrix, origs).squeeze()
        seed_matrix.name = 'Trips'
        
        dist_band_percs = tlds.xs([purp, md], level=['Purpose', 'Mode']).squeeze()
        bins = get_distance_bins(dist_band_percs)

        seed_matrix_banded = append_distance_band(seed_matrix, distance_matrix, bins)
        seed_matrix_banded.drop('Distance_miles', axis=1, inplace=True)

        # Convert percentages to trips
        dist_band_trips = dist_band_percs.mul(seed_matrix_banded.Trips.sum())
        dist_band_trips.name = 'Trips'

        # Convert df's index to IntervalIndex
        dist_band_trips = to_interval_index(dist_band_trips)

        dimensions = [['orig_zoneID'], ['dest_zoneID'], ['DistanceBand']]
        aggregates = [origs, dests, dist_band_trips]
        
        for cr in conv_rates:
            df_furnessed, converged = furness(seed_matrix_banded, dims=dimensions, aggrs=aggregates, convergence_rate=cr,
                                              max_iteration=50, weight_col='Trips', verbose=1)

            
            ipfn_results.setdefault('matrix', {})[k] = df_furnessed.copy()
            ipfn_results.setdefault('origs', {})[k] = origs.copy()
            ipfn_results.setdefault('dests', {})[k] = dests.copy()
            ipfn_results.setdefault('dests_factor', {})[k] = dests_factor.copy()
            
            if converged==1:
                ipfn_results.setdefault('conv_rate', {})[k] = cr
                print('{}:{}'.format(k, cr))
                break
            else:
                ipfn_results.setdefault('conv_rate', {})[k] = -1
                print('{}:-1'.format(k))

print('end')

ipfn converged
('HBO', 'Bus', 'AM'):0.05
ipfn converged
('HBO', 'Bus', 'IP'):0.05
ipfn converged
('HBO', 'Bus', 'OP'):0.05
ipfn converged
('HBO', 'Bus', 'PM'):0.05
Origs/Dests factor: 0.8469248597074569
ipfn converged
('HBO', 'CarDriver', 'AM'):0.05
ipfn converged
('HBO', 'CarDriver', 'IP'):0.05
Origs/Dests factor: 1.0799302470265013
ipfn converged
('HBO', 'CarDriver', 'OP'):0.05
ipfn converged
('HBO', 'CarDriver', 'PM'):0.05
Origs/Dests factor: 1.1747532678064179
ipfn converged
('HBO', 'CarPass', 'AM'):0.05
Origs/Dests factor: 0.9482016209674299
ipfn converged
('HBO', 'CarPass', 'IP'):0.05
ipfn converged
('HBO', 'CarPass', 'OP'):0.05
Origs/Dests factor: 0.931985916759954
ipfn converged
('HBO', 'CarPass', 'PM'):0.05
ipfn converged
('HBW', 'Bus', 'AM'):0.05
ipfn converged
('HBW', 'Bus', 'IP'):0.05
ipfn converged
('HBW', 'Bus', 'OP'):0.05
ipfn converged
('HBW', 'Bus', 'PM'):0.05
ipfn converged
('HBW', 'CarDriver', 'AM'):0.05
ipfn converged
('HBW', 'CarDriver', 'IP'):0.05
ipfn converged
(

In [23]:
results = ipfn_results['matrix'].copy()
dfs = []

for k, item in results.items():
    df = item.copy()
    purp , md, tp = k
    df.drop('DistanceBand', axis=1, inplace=True)
    df.insert(2, 'TimePeriod', tp)
    df.insert(2, 'Mode', md)
    df.insert(2, 'Purpose', purp)
    
    dfs.append(df)
    
OD_mats = pd.concat(dfs)
idx_cols = ['orig_zoneID', 'dest_zoneID', 'Purpose', 'Mode', 'TimePeriod']
OD_mats.set_index(idx_cols, inplace=True)

## Export

In [24]:
outfolder = r'../../MATSim/Synthetic Pop/Building/inputs'
outfile = 'ODs_purpose_mode_timeperiod.csv'
outfilepath = os.path.join(outfolder, outfile)

In [25]:
OD_mats.to_csv(outfilepath)